In [2]:
# 기존에 이미 splitter를 사용해서 나뉘어진 document가 글자 수, 문단 등과 같은 기준으로 나누어져 있음, 긴 맥락을 가진 경우에는 그 전체 맥락을 파악하기 어려울 수 있음
# 기존 벡터를 저장한 정보를 요약해서 추가로 벡터 디비에 저장
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

embeddings = OpenAIEmbeddings(model = 'text-embedding-3-large')
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory='./house' , # 벡터 디비를 저장할 경로
    collection_name='vector_db', # db name 같은거
) 

llm = ChatOpenAI(model = 'gpt-4.1-nano')
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 주어진 문장에 대해서 한국어로 요약하는 전문가입니다.'),
    ('user', '주어진 문서를 3문장으로 요약해주세요. 각 문장은 불릿 포인트 형식으로 작성해주세요. \n\n 주어진 문서 \n{docs}')
])

chain = prompt | llm | output_parser
retriever = vector_store.as_retriever()
all_docs = vector_store.get()
